In [21]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [22]:
IMAGE_SIZE = [224, 224]

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
train_path = '/content/drive/MyDrive/Dataset'
test_path = '/content/drive/MyDrive/test'

In [25]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

for layer in vgg.layers:
    layer.trainable = False

In [26]:
folders = glob('/content/drive/MyDrive/Dataset/*')

In [27]:
x = Flatten()(vgg.output)

In [28]:
prediction = Dense(len(folders), activation='softmax')(x)

In [29]:
model = Model(inputs = vgg.input, outputs = prediction)

In [30]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [31]:
model.compile(
    loss = 'binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [32]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                  shear_range = 0.2,
                                  zoom_range = 0.2,
                                  horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255,
                                  shear_range = 0.2,
                                  zoom_range = 0.2)

training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Dataset',
                                                target_size = (224, 224),
                                                batch_size = 32,
                                                class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/test',
                                           target_size = (224,224),
                                           batch_size = 32,
                                           class_mode = 'categorical')

Found 8823 images belonging to 2 classes.
Found 946 images belonging to 2 classes.


In [33]:
from datetime import datetime
from keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint(filepath='mymodel2.h5', verbose = 2, save_best_only=True)

callbac = [checkpoint]

start = datetime.now()

model_history = model.fit(
    training_set,
    validation_data = test_set,
    epochs = 50,
    steps_per_epoch = len(training_set),
    validation_steps = len(test_set),
    callbacks=callbac, verbose=2
    )

duration = datetime.now() - start
print("Time taken;", duration)

Epoch 1/50

Epoch 1: val_loss improved from inf to 0.31554, saving model to mymodel2.h5
276/276 - 1205s - loss: 0.3912 - accuracy: 0.8244 - val_loss: 0.3155 - val_accuracy: 0.8700 - 1205s/epoch - 4s/step
Epoch 2/50

Epoch 2: val_loss did not improve from 0.31554
276/276 - 215s - loss: 0.2845 - accuracy: 0.8786 - val_loss: 0.3518 - val_accuracy: 0.8457 - 215s/epoch - 780ms/step
Epoch 3/50

Epoch 3: val_loss did not improve from 0.31554
276/276 - 238s - loss: 0.2415 - accuracy: 0.9017 - val_loss: 0.3216 - val_accuracy: 0.8636 - 238s/epoch - 861ms/step
Epoch 4/50

Epoch 4: val_loss did not improve from 0.31554
276/276 - 220s - loss: 0.2171 - accuracy: 0.9127 - val_loss: 0.3748 - val_accuracy: 0.8362 - 220s/epoch - 798ms/step
Epoch 5/50

Epoch 5: val_loss did not improve from 0.31554
276/276 - 220s - loss: 0.2028 - accuracy: 0.9191 - val_loss: 0.6493 - val_accuracy: 0.7622 - 220s/epoch - 799ms/step
Epoch 6/50

Epoch 6: val_loss improved from 0.31554 to 0.29840, saving model to mymodel2.h5


UnknownError: ignored

In [19]:
import os
import cv2

training_set_path = "/content/drive/MyDrive/test/Real"  # Replace with the actual path to your training set

# Iterate over all files in the training set directory
for filename in os.listdir(training_set_path):
    image_path = os.path.join(training_set_path, filename)

    # Try to open the image using OpenCV
    try:
        image = cv2.imread(image_path)
        if image is None:
            # Image cannot be opened
            print(f"Deleting image: {filename}")
            os.remove(image_path)
    except Exception as e:
        # An error occurred while opening the image
        print(f"Error opening image {filename}: {str(e)}")